In [3]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from pathlib import Path
import pandas as pd

In [4]:
# Load training data,
df = pd.read_csv('resource/train.csv')
df.shape
df.head()

,filename,category
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,3
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,3
2,f6c172096818c5fab10ecae722840798.jpg,3
3,251ffd610399ac00fea7709c642676ee.jpg,3
4,73c7328b8eda399199fdedec6e4badaf.jpg,3


In [12]:
#Image Paths
from pathlib import Path

pathlist = Path('resource/train').glob('**/*.jpg')
pathlistDict = {}
for path in pathlist:
     # because path is object not string
     path_in_str = str(path)
     imgName = path_in_str.rsplit('/',1)[1]
     pathlistDict[imgName] = path_in_str

df['path'] = df['filename'].map(pathlistDict)
maxCategory = df.loc[df['category'].idxmax()]
maxCategory

# sampled_df = df.sample(frac=0.5, random_state=1)
# sampled_df.head()

In [1]:
images = []
for idx, row in df.iterrows():
    path = pathlistDict[row.filename]
    image = load_img(path)
    # x = img_to_array(image)
    images.append(image)
print(images[0])

NameError: name 'df' is not defined

In [5]:
# MODEL
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150), data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])